In [1]:
import zipfile
from os import listdir
from os.path import isfile, join
from pathlib import Path

from tqdm import tqdm

In [28]:
class gameObject:
    def __init__(self, objectBytes):
        xPosition = objectBytes[:4]
        yPosition = objectBytes[4:8]
        unUsed = objectBytes[8:10]
        objectWidth = objectBytes[10]
        objectHeight = objectBytes[11]
        objectFlags = objectBytes[12:16]
        childObjctFlags = objectBytes[16:20]
        extendedData = objectBytes[20:24]
        objectType = objectBytes[24:26]
        childobjectType = objectBytes[26:28]
        linkID = objectBytes[28:30]
        soundEffect = objectBytes[30:32]

In [29]:
class FreestandingSoundEffect:
    def __init__(self, soundBytes):
        effectType = soundBytes[0]
        xPosition = soundBytes[1]
        yPosition = soundBytes[2]
        padding = soundBytes[3]

In [30]:
class SnakeNode:
    def __init__(self, snakeNodeBytes):
        index = snakeNodeBytes[:1] #parte da zero
        mistery = snakeNodeBytes[1:3]
        zerocento = snakeNodeBytes[3:5]
        unused = snakeNodeBytes[5:7]   

In [31]:
class SnakeBlockTrack:
    def __init__(self, snakeBytes):
        snakeID = snakeBytes[0] #sarà 0 1 2 3 4?
        xPosition = snakeBytes[1]
        yPosition = snakeBytes[2]
        padding = snakeBytes[3]    

In [32]:
class ClearPipeNode:
    def __init__(self, clearPipeNodesBytes):
        unknown011 = clearPipeNodesBytes[0]
        index = clearPipeNodesBytes[1]
        unKnowCoordinates = clearPipeNodesBytes[2]
        unKnowCoordinates2 = clearPipeNodesBytes[3]
        always2 = clearPipeNodesBytes[4]
        

In [ ]:
class Rail:
    def __init__(self, railBytes):
        alwaysZero = railBytes[:1]
        alwaysZeroOne = railBytes[1]
        xPosition = railBytes[2]
        yPosition = railBytes[3]
        unknown014 = railBytes[4]
        unknown1 = railBytes[5:7]
        unknown2 = railBytes[7:9]
        unknown3 = railBytes[9:11]

In [33]:
class Icicle:
    def __init__(self, icicleBytes):
        xPosition = icicleBytes[0]
        yPosition = icicleBytes[1]
        zeroOne = icicleBytes[2]
        padding = icicleBytes[3]

In [22]:
class LevelArea:
    def __init__(self, levelBytes):
        courseTheme = levelBytes[0]
        autoScrollType = levelBytes[1]
        unknowHorientation = levelBytes[2]
        levelHorientation = levelBytes[3]
        lavaWaterHeight = levelBytes[4]
        lavaWaterMode = levelBytes[5]
        lavaWaterSpeed = levelBytes[6]
        minimunLavaWaterHeight = levelBytes[7]
        rightBoundary = levelBytes[8:12]
        topBoundary = levelBytes[12:16]
        leftBoundary = levelBytes[16:20]
        bottomBoundary = levelBytes[20:24]
        bitField = levelBytes[24:28]
        objectCount = levelBytes[28:32]
        freeStandingSoundEffectCount = levelBytes[32:36]
        snakeBlockCount = levelBytes[36:40]
        clearPipeCount = levelBytes[40:44]
        piranhaCreeperCount = levelBytes[44:48]
        expandingBlockCount = levelBytes[48:52]
        trackBlockCount = levelBytes[52:56]
        alwaysZero = levelBytes[56:60]
        tileCount = levelBytes[60:64]
        railCount = levelBytes[64:68]
        icicleCount = levelBytes[68:72]
        objects = levelBytes[72:83272]
        soundEffects = levelBytes[83272:84472]
        snake = levelBytes[84472:89292]
        clearPipe = levelBytes[89292:147692]
        creeper = levelBytes[147692:148532]
        expandingBlock = levelBytes[148532:148972]
        trackBlock = levelBytes[148972:149412]
        tiles = levelBytes[149412:165412]
        rail = levelBytes[165412:183412]
        iCicle = levelBytes[183412:184612]

In [2]:
levelFolder = Path("levels/")

In [3]:
levelsDownloaded = [f for f in listdir(levelFolder) if isfile(join(levelFolder, f))]

In [4]:
archive = zipfile.ZipFile(join(levelFolder, '007_-_bond_mario_bond__2958.zip'), 'r')
leveldata = archive.read('course_data_000.bcd')

In [5]:
leveldata[15215]

96

In [6]:
with open('tools/tmp/course_data_000.bcd', 'wb') as f:
    f.write(leveldata)

In [7]:
!.\tools\smm2dec.exe -h .\tools\tmp\course_data_000.bcd .\tools\tmp\course_data_dec.bcd

Decrypting course .\tools\tmp\course_data_000.bcd to .\tools\tmp\course_data_dec.bcd...
Done!


In [8]:
with open('tools/tmp/course_data_dec.bcd', 'rb') as f:
    leveldata = f.read()

In [9]:
leveldata[15215]

0

In [10]:
AESIV = leveldata[:16]  #array
randomiserState = leveldata[17:33]
AESCMAC = leveldata[34:50]


gameVersion = leveldata[:4]
gameVersion2 = leveldata[4:6]
padding = leveldata[6:8]
CRC32 = leveldata[8:12]
SCDL = leveldata[12:16]
header = leveldata[16:528]
mainLevelArea = leveldata[528:188656]
subLevelArea = leveldata[188656:376784]
CryptoCfg = leveldata[376784:]#non ne sono sicuro


In [11]:
CryptoCfg

b"\xf8T\x028'\xae\x05N\x07\xf9 \xeb\x84\xe8n\x1c'\xae\x05N\xf8T\x028\x84\xe8n\x1c\x07\xf9 \xeb\xe0\x18<=\x8aZ\xf5F0\xcd\xc6r\n<\xe1\xaf"

In [12]:
startY = header[0]
endY = header[1]
goalXx100 = header[2:4]
timeLimit = header[4:6]
targetAmountClearCondition = header[6:8]
year = header[8:10]
month = header[10]
day = header[11]
hour = header[12]
minute = header[13]
unknow = header[14]
clearConditionType = header[15]
clearConditionObject = header[16:20]
unknow4 = header[20:24]
levelFlags = header[24:28]
clearCheckTime = header[28:32]
randomValue = header[32:36]
courseWorld = header[36:40]

unknow2 = header[240]
gameStyle = header[241:244]
courseName = header[244:310]#wchar16
courseDescription = header[310:378]

In [13]:
courseTheme = mainLevelArea[0]
autoScrollType = mainLevelArea[1]
unknowHorientation = mainLevelArea[2]
levelHorientation = mainLevelArea[3]
lavaWaterHeight = mainLevelArea[4]
lavaWaterMode = mainLevelArea[5]
lavaWaterSpeed = mainLevelArea[6]
minimunLavaWaterHeight = mainLevelArea[7]
rightBoundary = mainLevelArea[8:12]
topBoundary = mainLevelArea[12:16]
leftBoundary = mainLevelArea[16:20]
bottomBoundary = mainLevelArea[20:24]
bitField = mainLevelArea[24:28]
objectCount = mainLevelArea[28:32]
freeStandingSoundEffectCount = mainLevelArea[32:36]
snakeBlockCount = mainLevelArea[36:40]
clearPipeCount = mainLevelArea[40:44]
piranhaCreeperCount = mainLevelArea[44:48]
expandingBlockCount = mainLevelArea[48:52]
trackBlockCount = mainLevelArea[52:56]
alwaysZero = mainLevelArea[56:60]
tileCount = mainLevelArea[60:64]
railCount = mainLevelArea[64:68]
icicleCount = mainLevelArea[68:72]
objects = mainLevelArea[72:83272]
soundEffects = mainLevelArea[83272:84472]
snake = mainLevelArea[84472:89292]
clearPipe = mainLevelArea[89292:147692]
creeper = mainLevelArea[147692:148532]
expandingBlock = mainLevelArea[148532:148972]
trackBlock = mainLevelArea[148972:149412]
tiles = mainLevelArea[149412:165412]
rail = mainLevelArea[165412:183412]
iCicle = mainLevelArea[183412:184612]

In [14]:
courseTheme

5

In [15]:
int.from_bytes(tileCount,"little", signed = False)

1128

In [16]:
gameVersion2

b'\x10\x00'

In [17]:
month

7

In [18]:
courseName.decode("utf-16")

'007 - Bond, Mario Bond ♪\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [19]:
courseDescription.decode("utf-16")

'Music and Play!\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [20]:
gameStyle.decode("utf-8")

'M3\x00'